# Roots and Integration of real-valued functions

Discussão sobre a implementação computacional e conceitos relevantes de métodos para encontrar a raíz de funções com valor real:

Discuss the relevant concepts and the computational implementation of methods to find roots of real-valued functions: 
- Bisection - Método da Bissecção
- Newton's Method (or Newton-Raphson) - Método de Newton-Raphson
- Quasi-Newton (Secant) - Método da Secante

In [ ]:
import matplotlib.pyplot as plt
x = [x/10 for x in range(0, 21)]
y = [x**3-1 for x in x]
plt.plot(x, y, 'b')
plt.axhline(y=0, linewidth=.5, color='k', marker='.')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(-3, 3)
y = [x**3-1-3*x+1 for x in x]
plt.plot(x, y, 'b')
plt.axhline(y=0, linewidth=.5, color='k', marker='.')
plt.show()

## Bracketing
### Bisection - Bissecsção

In [ ]:
def bisection(f, a, b, delta=1e-5, eps=1e-7):

    # f: a function
    # a: lower limit
    # b: upper limit
    
    if f(a)*f(b) <= 0:
        
        x_low = a
        x_high = b
        e = 1 # abs(x_high - x_low)
        d = 1 # f(M)
        i = 0
        #print("{joao:2d}: [{x_low:0.8f}, {x_high:0.8f}]".format(joao = i, x_low = x_low, x_high = x_high))
        
        while abs(e)>eps*(1+abs(x_low)+abs(x_high)) or abs(d)>delta:
            
            i += 1
            
            midpoint = (x_low+x_high)/2
            if f(x_low)*f(midpoint)<0:
                x_high = midpoint
            else:
                x_low = midpoint
                
            midpoint = (x_low+x_high)/2
            e = x_high-x_low
            d = f(midpoint)
            #print("{i:2d}: [{x_low:0.12f}, {x_high:0.12f}]".format(i = i, x_low = x_low, x_high = x_high))
            
        return midpoint

In [ ]:
def func(x):
    return x**3-1

In [ ]:
%time bisection(func, .4, 3)

In [ ]:
# testing
%time bisection(lambda x: x**3-1, .4, 3)

## Newton's Method - Método de Newton

http://mathworld.wolfram.com/NewtonsMethod.html <br>
http://tutorial.math.lamar.edu/Classes/CalcI/NewtonsMethod.aspx

In [ ]:
# derive
def derive(func, x, eps=1e-6):
    "derive: calculates the derivative of a real-valued function at a certain point of its domain."
    if func(x) or func(x) == 0:
        return (func(x+eps)-func(x-eps))/(2*eps)

In [ ]:
# newton
def newton(func, x, eps=10**(-5), delta=10**(-7)):
    "newton: finds a root of a real-valued function using Newton's method."
    
    e = 1
    d = abs(func(x))
    i = 0
    
    x0 = x
    
    while e>eps or d>delta:
        
        i = i+1
        x1 = x0-(func(x0)/derive(func, x0))
        e = abs(x1-x0)
        d = abs(func(x1))
        print("interaction: ", i)
        print("{0:.6f} -> {1:.6f}".format(x0,x1))
        print("e={0:.6f} d={1:.6f}".format(e,d))
        print("---------------------------------")
        x0 = x1
        
    return (x0, i)

In [ ]:
%time newton(lambda w: w**3-1, 3)

## Quasi-Newton

### Secant Method - Método Secante
http://mathworld.wolfram.com/SecantMethod.html

In [ ]:
# quasi-newton (secant)
def secant(func, x0, x1, eps=10**(-5), delta=10**(-7)):
    "quasi-newton (secant): finds a root of a real-valued function using the secant method."
    
    e = 1
    d = abs(func(x1))
    i = 0
    
    while e>eps or d>delta:
        
        i = i+1
        x2 = x1-(func(x1)*(x1-x0)/(func(x1)-func(x0)))
        e = abs(x2-x1)
        d = abs(func(x2))
        print("interaction: ", i)
        print("{0:.6f} -> {1:.6f}".format(x1, x2))
        print("e={0:.6f} d={1:.6f}".format(e, d))
        print("---------------------------------")
        x0 = x1
        x1 = x2
        
    return (x2, i)

In [ ]:
%time secant(lambda w: w**3-1, 4, 3)

## using `scipy.optimize`

In [ ]:
from scipy import optimize as opt

In [ ]:
%time opt.bisect(lambda w: w**3-1, .4, 3)

In [ ]:
%time opt.newton(lambda w: w**3-1, 3)

In [ ]:
%time opt.newton(lambda w: w**3-1, 3, fprime=lambda w: 3*(w**2))

## Application: Internal Rate of Return

In [ ]:
import matplotlib.pyplot as plt

v = [-1000, 500, 500, 500, -200]
def vpl(v, r):
        return sum([value*(1+r)**(-period) for period, value in enumerate(v)])

x = [x/400 for x in range(101)]
y = [vpl(v, r) for r in x]
plt.plot(x, y, 'b')
plt.axhline(y=0, linewidth=.5, color='k', marker='.')
plt.xlim((0,.3))
plt.show()

In [ ]:
from scipy import optimize as opt
def irr(v, r0):
    
    def vpl(r):
        return sum([value*(1+r)**(-t) for t,value in enumerate(v)])
    
    return opt.newton(vpl, r0)

In [ ]:
# testing
v = [-1000, 500, 500, 500, -200]
irr(v, 0.05)

## Integration

In [ ]:
import math

In [ ]:
print(dir(math))

In [ ]:
math.e

In [ ]:
math.exp(2)

In [ ]:
math.pi

In [ ]:
math.nan

### Rectangle method
$\int_{a}^{b}{f(x)dx} \approx \Delta x\,\sum_{i=1}^{n}{f(m_i)}$ where $\Delta x = \frac{b-a}{n}$ and $m_{i} = a + \frac{\Delta x}{2} i$

In [ ]:
def rectangle(f, a, b, n):
    
    step = (b-a)/n
    I = 0
    
    for i in range(int(n)):
        I += f(a+(2*i+1)*step/2)
        
    return I*step

In [ ]:
1e5

In [ ]:
# test 1
rectangle(lambda x: x**2, 0, 12, 1e5)

In [ ]:
# test 1
rectangle(lambda x: math.exp(x), 0, 1, 10)

### Trapezoid rule
$\int_{a}^{b}{f(x)dx} \approx \frac{\Delta x}{2}\,\sum_{i=1}^{n}{[f(x_{i-1})+f(x_{i})]} = \frac{\Delta x}{2}\,\left[f(x_{0})+2\,f(x_{1})+2\,f(x_{1})+...+2\,f(x_{n-1})+f(x_{n})\right]$ where $\Delta x = \frac{b-a}{n}$ and $x_{i} = a + \Delta x\,i$

In [ ]:
def trapezoid(f, a, b, n):
    
    step = (b - a)/n
    I = f(a) + f(b)
       
    for i in range(1, int(n), 1):
        I += 2*f(a+i*step)
                
    return I*step/2

In [ ]:
# test
trapezoid(lambda x: x**2, 0, 12, 1000)

In [ ]:
# test
trapezoid(lambda x: math.exp(x), 0, 1, 10)

### Simpson's rule
[Reference from Wolfran's MathWorld](http://mathworld.wolfram.com/SimpsonsRule.html)

$\int_{a}^{b}{f(x)dx} \approx \frac{\Delta x}{3}\,\left[f(x_{0})+4\,f(x_{1})+2\,f(x_{2})+4\,f(x_{3})+2\,f(x_{4})+...+2\,f(x_{n-2})+4\,f(x_{n-1})+f(x_{n})\right]$ where $\Delta x = \frac{b-a}{n}$.

In [ ]:
def simpson(f, a, b, n):
    
    step = (b-a)/n
    I = f(a)+f(b)
       
    for i in range(1, int(n), 2):
        I += 4*f(a+i*step)
    
    for i in range(2, int(n-1), 2):
        I += 2*f(a+i*step)
                
    return I*step/3

In [ ]:
# test
simpson(lambda x: x**2, 0, 12, 1e5)

In [ ]:
# test
simpson(lambda x: math.exp(x), 0, 1, 10)

In [47]:
from scipy import integrate
# help(integrate)

In [50]:
help(integrate.trapz)

Help on function trapz in module numpy.lib.function_base:

trapz(y, x=None, dx=1.0, axis=-1)
    Integrate along the given axis using the composite trapezoidal rule.
    
    Integrate `y` (`x`) along given axis.
    
    Parameters
    ----------
    y : array_like
        Input array to integrate.
    x : array_like, optional
        The sample points corresponding to the `y` values. If `x` is None,
        the sample points are assumed to be evenly spaced `dx` apart. The
        default is None.
    dx : scalar, optional
        The spacing between sample points when `x` is None. The default is 1.
    axis : int, optional
        The axis along which to integrate.
    
    Returns
    -------
    trapz : float
        Definite integral as approximated by trapezoidal rule.
    
    See Also
    --------
    sum, cumsum
    
    Notes
    -----
    Image [2]_ illustrates trapezoidal rule -- y-axis locations of points
    will be taken from `y` array, by default x-axis distances betwee